In [8]:
# encoding=utf8

# Pandas


In [9]:
import pandas as pd

# Parsing Unix timestamps

It's not obvious how to deal with Unix timestamps in pandas -- it took me quite a while to figure this out. The file we're using here is a popularity-contest file I found on my system at `/var/log/popularity-contest`.

Here's an [explanation of how this file works](http://popcon.ubuntu.com/README).

I'm going to hope that nothing in it is sensitive :)

In [10]:
# Read it, and remove the last row
popcon = pd.read_csv('../../DataSets/pybk030master/popularity-contest', sep=' ')[:-1]
popcon.columns = ['atime', 'ctime', 'package-name', 'mru-program', 'tag']

The colums are the access time, created time, package name, recently used program, and a tag

In [11]:
popcon[:5]

,atime,ctime,package-name,mru-program,tag
0,1387295797,1367633260,perl-base,/usr/bin/perl,NaN
1,1387295796,1354370480,login,/bin/su,NaN
2,1387295743,1354341275,libtalloc2,/usr/lib/x86_64-linux-gnu/libtalloc.so.2.0.7,NaN
3,1387295743,1387224204,libwbclient0,/usr/lib/x86_64-linux-gnu/libwbclient.so.0,<RECENT-CTIME>
4,1387295742,1354341253,libselinux1,/lib/x86_64-linux-gnu/libselinux.so.1,NaN


The magical part about parsing timestamps in pandas is that numpy datetimes are already stored as Unix timestamps. So all we need to do is tell pandas that these integers are actually datetimes -- it doesn't need to do any conversion at all.

We need to convert these to ints to start:

In [12]:
popcon['atime'] = popcon['atime'].astype(int)
popcon['ctime'] = popcon['ctime'].astype(int)

Every numpy array and pandas series has a dtype -- this is usually `int64`, `float64`, or `object`. Some of the time types available are `datetime64[s]`, `datetime64[ms]`, and `datetime64[us]`. There are also `timedelta` types, similarly.

We can use the `pd.to_datetime` function to convert our integer timestamps into datetimes. This is a constant-time operation -- we're not actually changing any of the data, just how pandas thinks about it.

In [13]:
popcon['atime'] = pd.to_datetime(popcon['atime'], unit='s')
popcon['ctime'] = pd.to_datetime(popcon['ctime'], unit='s')

If we look at the dtype now, it's `<M8[ns]`. As far as I can tell `M8` is secret code for `datetime64`.

In [14]:
popcon['atime'].dtype

dtype('<M8[ns]')

So now we can look at our `atime` and `ctime` as dates!

In [15]:
popcon[:5]

,atime,ctime,package-name,mru-program,tag
0,2013-12-17 15:56:37,2013-05-04 02:07:40,perl-base,/usr/bin/perl,NaN
1,2013-12-17 15:56:36,2012-12-01 14:01:20,login,/bin/su,NaN
2,2013-12-17 15:55:43,2012-12-01 05:54:35,libtalloc2,/usr/lib/x86_64-linux-gnu/libtalloc.so.2.0.7,NaN
3,2013-12-17 15:55:43,2013-12-16 20:03:24,libwbclient0,/usr/lib/x86_64-linux-gnu/libwbclient.so.0,<RECENT-CTIME>
4,2013-12-17 15:55:42,2012-12-01 05:54:13,libselinux1,/lib/x86_64-linux-gnu/libselinux.so.1,NaN


Now suppose we want to look at all packages that aren't libraries.

First, I want to get rid of everything with timestamp 0. Notice how we can just use a string in this comparison, even though it's actually a timestamp on the inside? That is because pandas is amazing.

In [16]:
popcon = popcon[popcon['atime'] > '1970-01-01']

Now we can use pandas' magical string abilities to just look at rows where the package name doesn't contain 'lib'.

In [17]:
nonlibraries = popcon[~popcon['package-name'].str.contains('lib')]

In [18]:
nonlibraries.sort_values(by='ctime', ascending=False)[:10]

,atime,ctime,package-name,mru-program,tag
57,2013-12-17 04:55:39,2013-12-17 04:55:42,ddd,/usr/bin/ddd,<RECENT-CTIME>
450,2013-12-16 20:03:20,2013-12-16 20:05:13,nodejs,/usr/bin/npm,<RECENT-CTIME>
454,2013-12-16 20:03:20,2013-12-16 20:05:04,switchboard-plug-keyboard,/usr/lib/plugs/pantheon/keyboard/options.txt,<RECENT-CTIME>
445,2013-12-16 20:03:20,2013-12-16 20:05:04,thunderbird-locale-en,/usr/lib/thunderbird-addons/extensions/langpac...,<RECENT-CTIME>
396,2013-12-16 20:08:27,2013-12-16 20:05:03,software-center,/usr/sbin/update-software-center,<RECENT-CTIME>
449,2013-12-16 20:03:20,2013-12-16 20:05:00,samba-common-bin,/usr/bin/net.samba3,<RECENT-CTIME>
397,2013-12-16 20:08:25,2013-12-16 20:04:59,postgresql-client-9.1,/usr/lib/postgresql/9.1/bin/psql,<RECENT-CTIME>
398,2013-12-16 20:08:23,2013-12-16 20:04:58,postgresql-9.1,/usr/lib/postgresql/9.1/bin/postmaster,<RECENT-CTIME>
452,2013-12-16 20:03:20,2013-12-16 20:04:55,php5-dev,/usr/include/php5/main/snprintf.h,<RECENT-CTIME>
440,2013-12-16 20:03:20,2013-12-16 20:04:54,php-pear,/usr/share/php/XML/Util.php,<RECENT-CTIME>


In [19]:
nonlibraries.index

Int64Index([   0,    1,   17,   18,   19,   20,   21,   22,   23,   24,
            ...
            2079, 2085, 2086, 2087, 2088, 2089, 2090, 2092, 2093, 2095],
           dtype='int64', length=1216)

In [20]:
nonlibraries = nonlibraries.set_index('atime')

In [21]:
nonlibraries

,ctime,package-name,mru-program,tag
atime,,,,
2013-12-17 15:56:37,2013-05-04 02:07:40,perl-base,/usr/bin/perl,NaN
2013-12-17 15:56:36,2012-12-01 14:01:20,login,/bin/su,NaN
2013-12-17 15:55:33,2013-11-25 16:25:38,fingerprint-gui,/lib/security/pam_fingerprint-gui.so,NaN
2013-12-17 15:55:32,2012-12-01 05:53:57,dash,/bin/dash,NaN
2013-12-17 15:55:29,2012-12-01 05:54:37,popularity-contest,/usr/sbin/popularity-contest,NaN
2013-12-17 15:53:05,2012-12-01 05:54:16,mlocate,/usr/bin/updatedb.mlocate,NaN
2013-12-17 15:53:05,2013-11-25 16:00:20,dpkg,/usr/sbin/dpkg-statoverride,NaN
2013-12-17 15:53:05,2012-12-01 14:01:16,coreutils,/bin/true,NaN
2013-12-17 15:53:05,2012-12-01 05:54:15,util-linux,/usr/bin/ionice,NaN


In [22]:
nonlibraries.index

DatetimeIndex(['2013-12-17 15:56:37', '2013-12-17 15:56:36',
               '2013-12-17 15:55:33', '2013-12-17 15:55:32',
               '2013-12-17 15:55:29', '2013-12-17 15:53:05',
               '2013-12-17 15:53:05', '2013-12-17 15:53:05',
               '2013-12-17 15:53:05', '2013-12-17 15:52:54',
               ...
               '2011-11-08 10:47:49', '2011-08-16 11:10:34',
               '2011-08-12 20:06:55', '2011-07-28 20:05:14',
               '2011-07-03 11:22:03', '2011-04-30 20:36:36',
               '2011-04-30 15:07:31', '2010-12-12 02:51:56',
               '2010-10-15 16:41:50', '2010-03-06 14:44:18'],
              dtype='datetime64[ns]', name='atime', length=1216, freq=None)

In [23]:
nonlibraries[nonlibraries.index.hour>15]

,ctime,package-name,mru-program,tag
atime,,,,
2013-12-16 23:44:52,2012-12-01 05:54:14,genisoimage,/usr/bin/mkisofs,NaN
2013-12-16 23:30:18,2013-07-29 22:35:24,google-talkplugin,/usr/lib/xulrunner/plugins/libnpgtpo3dautoplug...,NaN
2013-12-16 23:30:18,2012-12-01 14:04:13,totem-mozilla,/usr/lib/mozilla/plugins/libtotem-mully-plugin.so,NaN
2013-12-16 23:30:18,2013-04-29 02:16:27,gnome-shell,/usr/lib/mozilla/plugins/libgnome-shell-browse...,NaN
2013-12-16 23:30:18,2013-07-04 13:32:47,rhythmbox-mozilla,/usr/lib/mozilla/plugins/librhythmbox-itms-det...,NaN
2013-12-16 22:33:30,2013-05-04 02:07:24,bash,/bin/rbash,NaN
2013-12-16 22:31:12,2012-12-13 04:32:07,gdb,/usr/bin/gdb,NaN
2013-12-16 22:18:32,2013-01-27 18:25:00,p7zip-full,/usr/lib/p7zip/7z,NaN
2013-12-16 21:35:23,2013-05-10 02:49:43,gnome-keyring,/usr/lib/gnome-keyring/gnome-keyring-prompt,NaN


In [24]:
nonlibraries.index.weekofyear

array([51, 51, 51, ..., 49, 41,  9], dtype=int32)

In [25]:
nonlibraries.index.dayofweek

array([1, 1, 1, ..., 6, 4, 5], dtype=int32)

In [26]:
nonlibraries.index.daysinmonth

array([31, 31, 31, ..., 31, 31, 31], dtype=int32)

The whole message here is that if you have a timestamp in seconds or milliseconds or nanoseconds, then you can just "cast" it to a `'datetime64[the-right-thing]'` and pandas/numpy will take care of the rest.

<style>
    @font-face {
        font-family: "Computer Modern";
        src: url('http://mirrors.ctan.org/fonts/cm-unicode/fonts/otf/cmunss.otf');
    }
    div.cell{
        width:800px;
        margin-left:16% !important;
        margin-right:auto;
    }
    h1 {
        font-family: Helvetica, serif;
    }
    h4{
        margin-top:12px;
        margin-bottom: 3px;
       }
    div.text_cell_render{
        font-family: Computer Modern, "Helvetica Neue", Arial, Helvetica, Geneva, sans-serif;
        line-height: 145%;
        font-size: 130%;
        width:800px;
        margin-left:auto;
        margin-right:auto;
    }
    .CodeMirror{
            font-family: "Source Code Pro", source-code-pro,Consolas, monospace;
    }
    .text_cell_render h5 {
        font-weight: 300;
        font-size: 22pt;
        color: #4057A1;
        font-style: italic;
        margin-bottom: .5em;
        margin-top: 0.5em;
        display: block;
    }
    
    .warning{
        color: rgb( 240, 20, 20 )
        }  